In [11]:
from qiskit.opflow import Z, I
from qiskit.circuit.library import EfficientSU2
import numpy as np
from qiskit.algorithms.optimizers import SPSA
from qiskit_emulator import EmulatorProvider
import os
from time import sleep
from datetime import datetime, timedelta

num_qubits = 4
hamiltonian = (Z ^ Z) ^ (I ^ (num_qubits - 2))
target_energy = -1


# the rotation gates are chosen randomly, so we set a seed for reproducibility
ansatz = EfficientSU2(num_qubits, reps=1, entanglement='linear', insert_barriers=True)
# ansatz.draw('mpl', style='iqx')

optimizer = SPSA(maxiter=50)

np.random.seed(10)  # seed for reproducibility
initial_point = np.random.random(ansatz.num_parameters)

In [12]:
intermediate_info = {
    'nfev': [],
    'parameters': [],
    'energy': [],
    'stddev': []
}

timestamps = []

def raw_callback(*args):
    # print(args)
    (nfev, parameters, energy, stddev) = args[0]
    intermediate_info['nfev'].append(nfev)
    intermediate_info['parameters'].append(parameters)
    intermediate_info['energy'].append(energy)
    intermediate_info['stddev'].append(stddev)
#     timestamps.append(datetime.now())
    print(intermediate_info)
    
vqe_inputs = {
    'ansatz': ansatz,
    'operator': hamiltonian,
    'optimizer': {'name': 'SPSA', 'maxiter': 15},  # let's only do a few iterations!
    'initial_point': initial_point,
    'measurement_error_mitigation': True,
    'shots': 1024,
}

In [13]:
provider = EmulatorProvider()
provider.remote(os.getenv("SERVER_URL"))

In [14]:
program_id = provider.runtime.upload_program("vqe.py", description="Variational Quantum Eigensolver Program")

In [15]:
job = provider.runtime.run(
    program_id=program_id,
    inputs=vqe_inputs,
    options=None,
    callback=raw_callback
)

print('Job ID:', job.job_id)

Job ID: 10325f5bd43b


In [16]:
result = job.result()
while not result:
    print('no result yet.')
    sleep(0.5)
    result = job.result()

no result yet.
no result yet.
no result yet.
no result yet.
no result yet.
no result yet.
no result yet.
no result yet.
no result yet.
no result yet.
no result yet.
no result yet.
no result yet.
no result yet.
no result yet.
no result yet.
no result yet.
no result yet.
no result yet.
no result yet.
no result yet.
no result yet.
no result yet.
{'nfev': [2], 'parameters': [{'__type__': 'array', '__value__': [0.671320643266746, 0.1207519493594015, 0.5336482349262754, 0.8488038825386118, 0.5985070123025904, 0.32479664553084764, 0.29806286475962396, 0.8605307121989587, 0.06911083656253544, 0.18833981417401027, 0.7853598183677972, 0.8533933461949366, -0.09605173367208555, 0.41219226338577664, 0.7126209616521135, 0.7125260668293881]}], 'energy': [0.8144525208834245], 'stddev': [{'__type__': 'array', '__value__': {'__type__': 'complex', '__value__': [0.0181321939178912, 0.0]}}]}
{'nfev': [2, 3], 'parameters': [{'__type__': 'array', '__value__': [0.671320643266746, 0.1207519493594015, 0.5336482

no result yet.
no result yet.
no result yet.
no result yet.
no result yet.
no result yet.
{'nfev': [2, 3, 5, 6, 8, 9, 11, 12, 14, 15, 17, 18, 20, 21, 23, 24, 26, 27, 32], 'parameters': [{'__type__': 'array', '__value__': [0.671320643266746, 0.1207519493594015, 0.5336482349262754, 0.8488038825386118, 0.5985070123025904, 0.32479664553084764, 0.29806286475962396, 0.8605307121989587, 0.06911083656253544, 0.18833981417401027, 0.7853598183677972, 0.8533933461949366, -0.09605173367208555, 0.41219226338577664, 0.7126209616521135, 0.7125260668293881]}, {'__type__': 'array', '__value__': [0.871320643266746, -0.0792480506405985, 0.7336482349262754, 0.6488038825386119, 0.39850701230259045, 0.12479664553084766, 0.09806286475962397, 0.6605307121989588, 0.2691108365625354, -0.011660185825989738, 0.5853598183677973, 1.0533933461949365, 0.10394826632791446, 0.6121922633857766, 0.9126209616521135, 0.5125260668293882]}, {'__type__': 'array', '__value__': [1.126267769204797, -0.5206724738660159, 0.9885953

In [17]:
print(f"Intermediate Results: {intermediate_info}")
print(f'Reached {result["optimal_value"]} after {result["optimizer_evals"]} evaluations.')
print('Available keys:', list(result.keys()))

Intermediate Results: {'nfev': [2, 3, 5, 6, 8, 9, 11, 12, 14, 15, 17, 18, 20, 21, 23, 24, 26, 27, 32, 33, 35, 36, 38, 39, 41, 42, 44, 45], 'parameters': [{'__type__': 'array', '__value__': [0.671320643266746, 0.1207519493594015, 0.5336482349262754, 0.8488038825386118, 0.5985070123025904, 0.32479664553084764, 0.29806286475962396, 0.8605307121989587, 0.06911083656253544, 0.18833981417401027, 0.7853598183677972, 0.8533933461949366, -0.09605173367208555, 0.41219226338577664, 0.7126209616521135, 0.7125260668293881]}, {'__type__': 'array', '__value__': [0.871320643266746, -0.0792480506405985, 0.7336482349262754, 0.6488038825386119, 0.39850701230259045, 0.12479664553084766, 0.09806286475962397, 0.6605307121989588, 0.2691108365625354, -0.011660185825989738, 0.5853598183677973, 1.0533933461949365, 0.10394826632791446, 0.6121922633857766, 0.9126209616521135, 0.5125260668293882]}, {'__type__': 'array', '__value__': [1.126267769204797, -0.5206724738660159, 0.9885953608643264, 0.39385675660056096, 